# Импорт

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from keras import Model, Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical

# Методы:

### Загрузка данных игр в судоку из таблицы

In [2]:
def load_data(nb_train=50000, nb_test=320, full=False):

    if full:
        sudokus = pd.read_csv('sudoku.csv').values
    else:
        sudokus = next(
            pd.read_csv('sudoku.csv', chunksize=(nb_train + nb_test))
        ).values
        
    quizzes, solutions = sudokus.T
    flatX = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9))
                      for flatten_grid in quizzes])
    flaty = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9))
                      for flatten_grid in solutions])
    
    return (flatX[:nb_train], flaty[:nb_train]), (flatX[nb_train:], flaty[nb_train:])

### Рассчет точности решений

In [3]:
def diff(grids_true, grids_pred):
    return (grids_true != grids_pred).sum((1, 2))

### Удаление случайных ячеек в решенных играх

In [4]:
def delete_digits(X, n_delet=1):
    grids = X.argmax(3)
    for grid in grids:
        grid.flat[np.random.randint(0, 81, n_delet)] = 0
        
    return to_categorical(grids)

### Поиск решения

In [5]:
def batch_smart_solve(grids, solver):
    grids = grids.copy()
    for _ in range((grids == 0).sum((1, 2)).max()):
        preds = np.array(solver.predict(to_categorical(grids)))
        probs = preds.max(2).T
        values = preds.argmax(2).T + 1
        zeros = (grids == 0).reshape((grids.shape[0], 81))

        for grid, prob, value, zero in zip(grids, probs, values, zeros):
            if any(zero):
                where = np.where(zero)[0]
                confidence_position = where[prob[zero].argmax()]
                confidence_value = value[confidence_position]
                grid.flat[confidence_position] = confidence_value
    return grids

# Подготовка данных к обучению

In [17]:
load_data(nb_train=50000, nb_test=320, full=False)

((array([[[0, 0, 4, ..., 2, 0, 9],
          [0, 0, 5, ..., 0, 0, 1],
          [0, 7, 0, ..., 0, 4, 3],
          ...,
          [6, 0, 0, ..., 1, 0, 5],
          [0, 0, 3, ..., 6, 9, 0],
          [0, 4, 2, ..., 3, 0, 0]],
  
         [[0, 4, 0, ..., 0, 5, 0],
          [1, 0, 7, ..., 9, 6, 0],
          [5, 2, 0, ..., 0, 0, 0],
          ...,
          [0, 9, 0, ..., 5, 4, 3],
          [6, 0, 0, ..., 7, 0, 0],
          [2, 5, 0, ..., 1, 0, 0]],
  
         [[6, 0, 0, ..., 3, 8, 4],
          [0, 0, 8, ..., 0, 7, 2],
          [0, 0, 0, ..., 0, 0, 5],
          ...,
          [3, 1, 0, ..., 0, 5, 0],
          [0, 8, 9, ..., 0, 0, 0],
          [5, 0, 2, ..., 1, 9, 0]],
  
         ...,
  
         [[0, 0, 8, ..., 0, 0, 7],
          [9, 0, 0, ..., 1, 0, 0],
          [0, 1, 0, ..., 4, 0, 0],
          ...,
          [0, 0, 0, ..., 9, 3, 0],
          [5, 9, 0, ..., 0, 0, 0],
          [0, 0, 3, ..., 6, 2, 1]],
  
         [[0, 0, 9, ..., 0, 0, 0],
          [4, 0, 0, ..., 7, 0, 8

In [18]:
input_shape = (9, 9, 10)
(_, ytrain), (Xtest, ytest) = load_data()

Xtrain = to_categorical(ytrain).astype('float32')
Xtest = to_categorical(Xtest).astype('float32')

ytrain = to_categorical(ytrain-1).astype('float32')
ytest = to_categorical(ytest-1).astype('float32')

# Определить модель keras

In [19]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=input_shape))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())

grid = Input(shape=input_shape)
features = model(grid)

digit_placeholders = [
    Dense(9, activation='softmax')(features)
    for i in range(81)
]

solver = Model(grid, digit_placeholders)
solver.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Итерационное обучение модели на тестовых данных

In [20]:
history = solver.fit(
    delete_digits(Xtrain, 0),
    [ytrain[:, i, j, :] for i in range(9) for j in range(9)],
    batch_size=64,
    epochs=1,
    verbose=1,
)

157/157 [==============================] - 97s 437ms/step - loss: 97.8747 - dense_85_loss: 1.2074 - dense_86_loss: 1.2112 - dense_87_loss: 1.1929 - dense_88_loss: 1.1940 - dense_89_loss: 1.2002 - dense_90_loss: 1.2026 - dense_91_loss: 1.1920 - dense_92_loss: 1.1963 - dense_93_loss: 1.1982 - dense_94_loss: 1.2010 - dense_95_loss: 1.2109 - dense_96_loss: 1.1985 - dense_97_loss: 1.2081 - dense_98_loss: 1.2228 - dense_99_loss: 1.2172 - dense_100_loss: 1.1972 - dense_101_loss: 1.2140 - dense_102_loss: 1.2238 - dense_103_loss: 1.2118 - dense_104_loss: 1.1981 - dense_105_loss: 1.2108 - dense_106_loss: 1.1984 - dense_107_loss: 1.2148 - dense_108_loss: 1.2148 - dense_109_loss: 1.2100 - dense_110_loss: 1.1970 - dense_111_loss: 1.2209 - dense_112_loss: 1.1949 - dense_113_loss: 1.2070 - dense_114_loss: 1.2117 - dense_115_loss: 1.2148 - dense_116_loss: 1.2120 - dense_117_loss: 1.2216 - dense_118_loss: 1.2080 - dense_119_loss: 1.2066 - dense_120_loss: 1.2047 - dense_121_loss: 1.2116 - dense_122_loss

## Решение судоку со случайно удаленными значениями

In [21]:
early_stop = EarlyStopping(patience=2, verbose=1)

i = 1
for nb_epochs, nb_delete in zip(
        #[1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [1, 1, 2, 3, 5, 5, 5],
        [1, 2, 5, 10, 20, 40, 55]
        #[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        #[1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55]
):
    print('Pass n° {} ...'.format(i))
    i += 1
    
    history = solver.fit(
        delete_digits(Xtrain, nb_delete),
        [ytrain[:, i, j, :] for i in range(9) for j in range(9)],
        validation_data=(
            delete_digits(Xtrain, nb_delete),
            [ytrain[:, i, j, :] for i in range(9) for j in range(9)]),
        batch_size=64,
        epochs=nb_epochs,
        verbose=1,
        callbacks=[early_stop]
    )

Pass n° 1 ...
157/157 [==============================] - 110s 700ms/step - loss: 3.8883 - dense_85_loss: 0.0460 - dense_86_loss: 0.0484 - dense_87_loss: 0.0475 - dense_88_loss: 0.0473 - dense_89_loss: 0.0488 - dense_90_loss: 0.0510 - dense_91_loss: 0.0436 - dense_92_loss: 0.0512 - dense_93_loss: 0.0483 - dense_94_loss: 0.0475 - dense_95_loss: 0.0448 - dense_96_loss: 0.0469 - dense_97_loss: 0.0508 - dense_98_loss: 0.0470 - dense_99_loss: 0.0500 - dense_100_loss: 0.0453 - dense_101_loss: 0.0482 - dense_102_loss: 0.0506 - dense_103_loss: 0.0471 - dense_104_loss: 0.0484 - dense_105_loss: 0.0498 - dense_106_loss: 0.0470 - dense_107_loss: 0.0481 - dense_108_loss: 0.0460 - dense_109_loss: 0.0531 - dense_110_loss: 0.0507 - dense_111_loss: 0.0459 - dense_112_loss: 0.0497 - dense_113_loss: 0.0472 - dense_114_loss: 0.0437 - dense_115_loss: 0.0465 - dense_116_loss: 0.0483 - dense_117_loss: 0.0493 - dense_118_loss: 0.0495 - dense_119_loss: 0.0470 - dense_120_loss: 0.0462 - dense_121_loss: 0.0502 - 

Pass n° 2 ...
157/157 [==============================] - 105s 670ms/step - loss: 2.6481 - dense_85_loss: 0.0311 - dense_86_loss: 0.0336 - dense_87_loss: 0.0288 - dense_88_loss: 0.0334 - dense_89_loss: 0.0323 - dense_90_loss: 0.0332 - dense_91_loss: 0.0312 - dense_92_loss: 0.0327 - dense_93_loss: 0.0321 - dense_94_loss: 0.0335 - dense_95_loss: 0.0353 - dense_96_loss: 0.0317 - dense_97_loss: 0.0320 - dense_98_loss: 0.0315 - dense_99_loss: 0.0316 - dense_100_loss: 0.0318 - dense_101_loss: 0.0295 - dense_102_loss: 0.0294 - dense_103_loss: 0.0361 - dense_104_loss: 0.0336 - dense_105_loss: 0.0292 - dense_106_loss: 0.0366 - dense_107_loss: 0.0313 - dense_108_loss: 0.0336 - dense_109_loss: 0.0333 - dense_110_loss: 0.0290 - dense_111_loss: 0.0340 - dense_112_loss: 0.0324 - dense_113_loss: 0.0380 - dense_114_loss: 0.0287 - dense_115_loss: 0.0336 - dense_116_loss: 0.0317 - dense_117_loss: 0.0323 - dense_118_loss: 0.0301 - dense_119_loss: 0.0322 - dense_120_loss: 0.0371 - dense_121_loss: 0.0347 - 

Pass n° 3 ...
Epoch 1/2
157/157 [==============================] - 106s 676ms/step - loss: 4.5888 - dense_85_loss: 0.0548 - dense_86_loss: 0.0521 - dense_87_loss: 0.0562 - dense_88_loss: 0.0503 - dense_89_loss: 0.0641 - dense_90_loss: 0.0597 - dense_91_loss: 0.0561 - dense_92_loss: 0.0520 - dense_93_loss: 0.0557 - dense_94_loss: 0.0494 - dense_95_loss: 0.0536 - dense_96_loss: 0.0589 - dense_97_loss: 0.0650 - dense_98_loss: 0.0596 - dense_99_loss: 0.0513 - dense_100_loss: 0.0602 - dense_101_loss: 0.0617 - dense_102_loss: 0.0582 - dense_103_loss: 0.0574 - dense_104_loss: 0.0570 - dense_105_loss: 0.0567 - dense_106_loss: 0.0565 - dense_107_loss: 0.0539 - dense_108_loss: 0.0581 - dense_109_loss: 0.0580 - dense_110_loss: 0.0632 - dense_111_loss: 0.0586 - dense_112_loss: 0.0526 - dense_113_loss: 0.0544 - dense_114_loss: 0.0596 - dense_115_loss: 0.0645 - dense_116_loss: 0.0554 - dense_117_loss: 0.0648 - dense_118_loss: 0.0555 - dense_119_loss: 0.0580 - dense_120_loss: 0.0540 - dense_121_loss:

Epoch 2/2
157/157 [==============================] - 104s 663ms/step - loss: 2.6682 - dense_85_loss: 0.0289 - dense_86_loss: 0.0342 - dense_87_loss: 0.0318 - dense_88_loss: 0.0299 - dense_89_loss: 0.0385 - dense_90_loss: 0.0317 - dense_91_loss: 0.0300 - dense_92_loss: 0.0295 - dense_93_loss: 0.0311 - dense_94_loss: 0.0311 - dense_95_loss: 0.0287 - dense_96_loss: 0.0365 - dense_97_loss: 0.0307 - dense_98_loss: 0.0355 - dense_99_loss: 0.0303 - dense_100_loss: 0.0355 - dense_101_loss: 0.0369 - dense_102_loss: 0.0385 - dense_103_loss: 0.0303 - dense_104_loss: 0.0357 - dense_105_loss: 0.0308 - dense_106_loss: 0.0320 - dense_107_loss: 0.0369 - dense_108_loss: 0.0294 - dense_109_loss: 0.0350 - dense_110_loss: 0.0300 - dense_111_loss: 0.0296 - dense_112_loss: 0.0303 - dense_113_loss: 0.0308 - dense_114_loss: 0.0374 - dense_115_loss: 0.0282 - dense_116_loss: 0.0346 - dense_117_loss: 0.0293 - dense_118_loss: 0.0329 - dense_119_loss: 0.0269 - dense_120_loss: 0.0371 - dense_121_loss: 0.0329 - dens

Pass n° 4 ...
Epoch 1/3
157/157 [==============================] - 72s 460ms/step - loss: 8.2514 - dense_85_loss: 0.1002 - dense_86_loss: 0.1042 - dense_87_loss: 0.0976 - dense_88_loss: 0.1048 - dense_89_loss: 0.1112 - dense_90_loss: 0.1019 - dense_91_loss: 0.1017 - dense_92_loss: 0.0912 - dense_93_loss: 0.0994 - dense_94_loss: 0.0973 - dense_95_loss: 0.0981 - dense_96_loss: 0.0991 - dense_97_loss: 0.1035 - dense_98_loss: 0.1069 - dense_99_loss: 0.1109 - dense_100_loss: 0.0990 - dense_101_loss: 0.1017 - dense_102_loss: 0.0972 - dense_103_loss: 0.0983 - dense_104_loss: 0.0964 - dense_105_loss: 0.1072 - dense_106_loss: 0.1088 - dense_107_loss: 0.0905 - dense_108_loss: 0.0970 - dense_109_loss: 0.0983 - dense_110_loss: 0.1127 - dense_111_loss: 0.0955 - dense_112_loss: 0.1041 - dense_113_loss: 0.1030 - dense_114_loss: 0.1025 - dense_115_loss: 0.1030 - dense_116_loss: 0.1066 - dense_117_loss: 0.1034 - dense_118_loss: 0.0906 - dense_119_loss: 0.0929 - dense_120_loss: 0.1012 - dense_121_loss: 

...

135/157 [========================>.....] - ETA: 4s - loss: 32.2248 - dense_85_loss: 0.3991 - dense_86_loss: 0.3987 - dense_87_loss: 0.3982 - dense_88_loss: 0.4101 - dense_89_loss: 0.3951 - dense_90_loss: 0.4110 - dense_91_loss: 0.3861 - dense_92_loss: 0.3799 - dense_93_loss: 0.3966 - dense_94_loss: 0.3977 - dense_95_loss: 0.3959 - dense_96_loss: 0.3945 - dense_97_loss: 0.3951 - dense_98_loss: 0.4008 - dense_99_loss: 0.4154 - dense_100_loss: 0.3922 - dense_101_loss: 0.3845 - dense_102_loss: 0.3895 - dense_103_loss: 0.3918 - dense_104_loss: 0.4041 - dense_105_loss: 0.4115 - dense_106_loss: 0.3999 - dense_107_loss: 0.4021 - dense_108_loss: 0.4043 - dense_109_loss: 0.4036 - dense_110_loss: 0.4011 - dense_111_loss: 0.3978 - dense_112_loss: 0.3952 - dense_113_loss: 0.3885 - dense_114_loss: 0.4038 - dense_115_loss: 0.3982 - dense_116_loss: 0.3941 - dense_117_loss: 0.4082 - dense_118_loss: 0.3965 - dense_119_loss: 0.3980 - dense_120_loss: 0.3965 - dense_121_loss: 0.3925 - dense_122_loss: 0.387

136/157 [========================>.....] - ETA: 4s - loss: 32.2356 - dense_85_loss: 0.4010 - dense_86_loss: 0.3987 - dense_87_loss: 0.3984 - dense_88_loss: 0.4111 - dense_89_loss: 0.3955 - dense_90_loss: 0.4116 - dense_91_loss: 0.3863 - dense_92_loss: 0.3803 - dense_93_loss: 0.3972 - dense_94_loss: 0.3980 - dense_95_loss: 0.3973 - dense_96_loss: 0.3948 - dense_97_loss: 0.3958 - dense_98_loss: 0.4008 - dense_99_loss: 0.4149 - dense_100_loss: 0.3927 - dense_101_loss: 0.3847 - dense_102_loss: 0.3895 - dense_103_loss: 0.3922 - dense_104_loss: 0.4041 - dense_105_loss: 0.4112 - dense_106_loss: 0.3993 - dense_107_loss: 0.4035 - dense_108_loss: 0.4040 - dense_109_loss: 0.4030 - dense_110_loss: 0.4009 - dense_111_loss: 0.3976 - dense_112_loss: 0.3952 - dense_113_loss: 0.3883 - dense_114_loss: 0.4032 - dense_115_loss: 0.3987 - dense_116_loss: 0.3929 - dense_117_loss: 0.4087 - dense_118_loss: 0.3966 - dense_119_loss: 0.3979 - dense_120_loss: 0.3970 - dense_121_loss: 0.3934 - dense_122_loss: 0.388

157/157 [==============================] - 67s 428ms/step - loss: 32.6473 - dense_85_loss: 0.4072 - dense_86_loss: 0.4035 - dense_87_loss: 0.4057 - dense_88_loss: 0.4101 - dense_89_loss: 0.4071 - dense_90_loss: 0.4143 - dense_91_loss: 0.3907 - dense_92_loss: 0.3895 - dense_93_loss: 0.4053 - dense_94_loss: 0.4047 - dense_95_loss: 0.4016 - dense_96_loss: 0.4021 - dense_97_loss: 0.4001 - dense_98_loss: 0.4099 - dense_99_loss: 0.4179 - dense_100_loss: 0.3969 - dense_101_loss: 0.3896 - dense_102_loss: 0.3973 - dense_103_loss: 0.3979 - dense_104_loss: 0.4068 - dense_105_loss: 0.4089 - dense_106_loss: 0.4117 - dense_107_loss: 0.4105 - dense_108_loss: 0.4079 - dense_109_loss: 0.4073 - dense_110_loss: 0.4059 - dense_111_loss: 0.4022 - dense_112_loss: 0.3956 - dense_113_loss: 0.3938 - dense_114_loss: 0.4059 - dense_115_loss: 0.3989 - dense_116_loss: 0.3920 - dense_117_loss: 0.4122 - dense_118_loss: 0.4054 - dense_119_loss: 0.4017 - dense_120_loss: 0.4002 - dense_121_loss: 0.4006 - dense_122_loss

Epoch 4/5
157/157 [==============================] - 70s 447ms/step - loss: 30.1470 - dense_85_loss: 0.3748 - dense_86_loss: 0.3751 - dense_87_loss: 0.3809 - dense_88_loss: 0.3734 - dense_89_loss: 0.3743 - dense_90_loss: 0.3743 - dense_91_loss: 0.3597 - dense_92_loss: 0.3617 - dense_93_loss: 0.3712 - dense_94_loss: 0.3703 - dense_95_loss: 0.3641 - dense_96_loss: 0.3715 - dense_97_loss: 0.3757 - dense_98_loss: 0.3809 - dense_99_loss: 0.3861 - dense_100_loss: 0.3695 - dense_101_loss: 0.3721 - dense_102_loss: 0.3676 - dense_103_loss: 0.3684 - dense_104_loss: 0.3780 - dense_105_loss: 0.3723 - dense_106_loss: 0.3851 - dense_107_loss: 0.3762 - dense_108_loss: 0.3754 - dense_109_loss: 0.3803 - dense_110_loss: 0.3776 - dense_111_loss: 0.3732 - dense_112_loss: 0.3517 - dense_113_loss: 0.3726 - dense_114_loss: 0.3653 - dense_115_loss: 0.3692 - dense_116_loss: 0.3610 - dense_117_loss: 0.3676 - dense_118_loss: 0.3663 - dense_119_loss: 0.3631 - dense_120_loss: 0.3783 - dense_121_loss: 0.3565 - dens

Epoch 00004: early stopping


# Результаты

In [ ]:
quizzes = Xtest.argmax(3)
true_grids = ytest.argmax(3) + 1
smart_guesses = batch_smart_solve(quizzes, solver)

deltas = diff(true_grids, smart_guesses)
accuracy = (deltas == 0).mean()

In [ ]:
print(
"""
Решено:\t {}
Корректных:\t {}
Точность:\t {}
""".format(
deltas.shape[0], (deltas==0).sum(), accuracy
)
)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()